## Sentinel-1 TOPS stack processor
The detailed algorithm for stack processing of TOPS data can be find here:

+ Fattahi, H., P. Agram, and M. Simons (2016), A Network-Based Enhanced Spectral Diversity Approach for TOPS Time-Series Analysis, IEEE Transactions on Geoscience and Remote Sensing, 55(2), 777-786, doi:[10.1109/TGRS.2016.2614925](https://ieeexplore.ieee.org/abstract/document/7637021).

-----------------------------------

The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

#### Be sure [default]  the ~/.netrc file has been changed to include valid credentials for urs.earthdata.nasa.gov.

In [1]:
def get_current_time():
    import datetime
    return datetime.datetime.now()
run_start_time = get_current_time()
print("PGE run start time : {}".format(run_start_time))


PGE run start time : 2021-03-09 20:27:00.745007


In [2]:
%%bash
pip install git+https://github.com/hysds/osaka.git#egg=osaka
pip install opencv-python-headless
conda install -c conda-forge fiona -y
conda install -c conda-forge parallel -y
source /opt/conda/etc/profile.d/conda.sh

  Cloning https://github.com/hysds/osaka.git to /tmp/pip-install-pdb2vwp_/osaka_f2d7e156a23f4559a2fde5db539f18af
Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
import os
import json
from math import floor, ceil
import json
import re
import osaka
import osaka.main
from builtins import str
import os, sys, re, json, logging, traceback, requests, argparse
from datetime import datetime
from pprint import pformat
from requests.packages.urllib3.exceptions import (InsecureRequestWarning,
                                                  InsecurePlatformWarning)
try: from html.parser import HTMLParser
except: from html.parser import HTMLParser
        
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecurePlatformWarning)
PROCESSING_START=datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
print(PROCESSING_START)

PGE_BASE=os.getcwd()
SLC_PATH = "zip"
print(PGE_BASE)

SLC_RE = re.compile(r'(?P<mission>S1\w)_IW_SLC__.*?' +
                    r'_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})' +
                    r'T(?P<start_hour>\d{2})(?P<start_min>\d{2})(?P<start_sec>\d{2})' +
                    r'_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})' +
                    r'T(?P<end_hour>\d{2})(?P<end_min>\d{2})(?P<end_sec>\d{2})_.*$')
ISCE_HOME="/opt/isce2/isce"

QC_SERVER = 'https://qc.sentinel1.eo.esa.int/'
DATA_SERVER = 'http://aux.sentinel1.eo.esa.int/'

ORBITMAP = [('precise','aux_poeorb', 100),
            ('restituted','aux_resorb', 100)]

OPER_RE = re.compile(r'S1\w_OPER_AUX_(?P<type>\w+)_OPOD_(?P<yr>\d{4})(?P<mo>\d{2})(?P<dy>\d{2})')


    

2021-03-09T20:27:46
/home/jovyan/topsstack-hamsar


In [4]:
from typing import List
ISCE_HOME="/opt/isce2/isce"
min_lat = 34.6002832
max_lat = 34.6502392
min_lon = -79.0801608
max_lon = -78.9705888
master_date = ""
localize_slcs: List[str] = ["S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664",
                      "S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_93D5",
                      "S1A_IW_SLC__1SDV_20150830T231332_20150830T231402_007499_00A5A9_02B3",
                      "S1A_IW_SLC__1SDV_20160414T231321_20160414T231350_010824_01030B_F02B",
                      "S1A_IW_SLC__1SDV_20160414T231348_20160414T231416_010824_01030B_9FA9"
                     ]

In [5]:
ctx = {}
ctx["min_lat"] = min_lat
ctx["max_lat"] = max_lat
ctx["min_lon"] = min_lon
ctx["max_lon"] = max_lon
ctx["master_date"]=""
ctx["localize_slcs"]= localize_slcs

print(json.dumps(ctx, indent=4))
wd = os.getcwd()
global runtime_dict
runtime_dict = {}

with open('_stdout.txt', 'w') as f:
    f.write("Output File")
with open('_context.json', 'w') as outfile:
    json.dump(ctx, outfile)

{
    "min_lat": 34.6002832,
    "max_lat": 34.6502392,
    "min_lon": -79.0801608,
    "max_lon": -78.9705888,
    "master_date": "",
    "localize_slcs": [
        "S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664",
        "S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_93D5",
        "S1A_IW_SLC__1SDV_20150830T231332_20150830T231402_007499_00A5A9_02B3",
        "S1A_IW_SLC__1SDV_20160414T231321_20160414T231350_010824_01030B_F02B",
        "S1A_IW_SLC__1SDV_20160414T231348_20160414T231416_010824_01030B_9FA9"
    ]
}


In [34]:
import requests
import re
from bs4 import BeautifulSoup
import os
from datetime import datetime, timedelta

def session_get(session, url):
    return session.get(url, verify=False)

def get_download_orbit_dict(slc_start_dt, slc_end_dt, mission_type, margin=60.0, datefmt="%Y%m%dT%H%M%S"):
    def is_orbit(href):
        return href and re.compile(r'^S1.*EOF$').search(href)
    download_orbit_dict = {}
    found = False
    for fidelity in ('POEORB', 'RESORB'):
        if fidelity == "POEORB":
            dts = [slc_start_dt + timedelta(days=20), slc_start_dt + timedelta(days=22)]
        else:
            dts = [slc_start_dt, slc_start_dt + timedelta(days=1)]
        for dt in dts:
            yy, mm, dd = "%04d" % dt.year, "%02d" % dt.month, "%02d" % dt.day
            url = f"http://aux.sentinel1.eo.esa.int/{fidelity}/{yy}/{mm}/{dd}/"
            #print(f"url: {url}")
            session = requests.Session()
            r = session_get(session, url)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, 'html.parser')
            for a in soup.find_all(href=is_orbit):
                #print(a['href'])
                m = re.search(r'^(?P<sat>S1[AB])_.*$', a['href'])
                sat = m.groupdict()['sat']
                if mission_type == sat:
                    orbit = os.path.basename(a['href'])
                    fields = orbit.split('_')
                    orbit_start_date_time = datetime.strptime(fields[6].replace('V',''), datefmt) + timedelta(seconds=margin)
                    orbit_stop_date_time = datetime.strptime(fields[7].replace('.EOF',''), datefmt) - timedelta(seconds=margin)
                    if slc_start_dt >= orbit_start_date_time and slc_end_dt < orbit_stop_date_time:
                        download_orbit_dict[os.path.splitext(os.path.basename(a['href']))[0]] = f"{url}{a['href']}"
                        found = True
                        break
            if found: break
    return download_orbit_dict

def get_orbit_files():
    from datetime import datetime, timedelta
    import json
    import os
    import osaka
    import osaka.main
    orbit_dates = []
    for slc in ctx["localize_slcs"]:
        match = SLC_RE.search(slc)
        if not match:
            raise RuntimeError("Failed to recognize SLC ID %s." %slc)
        mission = match.group('mission')
        slc_start_dt_str = "{}-{}-{}T{}:{}:{}".format(match.group('start_year'), 
                                                      match.group('start_month'),
                                                      match.group('start_day'),
                                                      match.group('start_hour'),
                                                      match.group('start_min'),
                                                      match.group('start_sec'))
        slc_start_dt = datetime.strptime(slc_start_dt_str, "%Y-%m-%dT%H:%M:%S")
        slc_end_dt_str = "{}-{}-{}T{}:{}:{}".format(match.group('end_year'), 
                                                      match.group('end_month'),
                                                      match.group('end_day'),
                                                      match.group('end_hour'),
                                                      match.group('end_min'),
                                                      match.group('end_sec'))
        slc_end_dt = datetime.strptime(slc_end_dt_str, "%Y-%m-%dT%H:%M:%S")
        day_dt_str = slc_start_dt.strftime('%Y-%m-%d')
        print(f"slc_start_dt_str: {slc_start_dt_str}")
        print(f"slc_end_dt_str: {slc_end_dt_str}")
        if day_dt_str not in orbit_dates:
            print(f"checking {day_dt_str}")
            orbit_dates.append(day_dt_str)
            orbit_dict = get_download_orbit_dict(slc_start_dt, slc_end_dt, mission)
            directory = os.path.join(wd, "orbits")
            if not os.path.exists(directory):
                os.makedirs(directory)
                print(f"created {directory}")
            for k, v in orbit_dict.items():
                osaka.main.get(v, directory)

    
def download_slc(slc_id, path):
    from urllib.parse import urlparse
    url = "https://datapool.asf.alaska.edu/SLC/SA/{}.zip".format(slc_id)
    print("Downloading {} : {}".format(slc_id, url))
    
    if not os.path.exists(path):
        os.makedirs(path)
        
    if not os.path.exists(os.path.join(path, os.path.basename(urlparse(url).path) ) ):               # Output: /kyle/09-09-201315-47-571378756077.jpg
        osaka.main.get(url, path)

def run_cmd_output(cmd):
    from subprocess import check_output, CalledProcessError
    cmd_line = " ".join(cmd)
    print("Calling: {}".format(cmd_line))
    output = check_output(cmd_line, shell=True)
    return output

def run_cmd(cmd):
    !source /opt/isce2/isce_env.sh
    !export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
    !export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}

    import subprocess
    from subprocess import check_call, CalledProcessError
    import sys
    print(cmd)
    cmd_line = " ".join(cmd)
    print("Calling : {}".format(cmd_line))
    p = subprocess.Popen(cmd_line, shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    while True: 
        line = p.stdout.readline()
        if not line:
            break
        print(line.strip())
        sys.stdout.flush()
        
def get_minimum_bounding_rectangle():
    from math import floor, ceil
    cwd = os.getcwd()
    slc_ids = [x for x in os.listdir('.') if os.path.isdir(x) and '_SLC__' in x]

    all_lats = []
    all_lons = []
    for slc in slc_ids:
        slc_met_json = slc + '.met.json'

        with open(os.path.join(cwd, slc, slc_met_json), 'r') as f:
            data = json.load(f)
            bbox = data['bbox']
            for coord in bbox:
                all_lats.append(coord[0])
                all_lons.append(coord[1])

    min_lat = min(all_lats) + 0.2
    max_lat = max(all_lats) - 0.1
    min_lon = min(all_lons)
    max_lon = max(all_lons)

    min_lat_lo = floor(min_lat)
    max_lat_hi = ceil(max_lat)
    min_lon_lo = floor(min_lon)
    max_lon_hi = ceil(max_lon)

    return min_lat, max_lat, min_lon, max_lon, min_lat_lo, max_lat_hi, min_lon_lo, max_lon_hi

def get_user_input_bbox(ctx_file):
    """
    :param ctx_file: dictionary from cxt file
    :return: void
    """
    from math import floor, ceil
    min_lat = ctx_file['min_lat']
    max_lat = ctx_file['max_lat']
    min_lon = ctx_file['min_lon']
    max_lon = ctx_file['max_lon']

    min_lat_lo = floor(min_lat)
    max_lat_hi = ceil(max_lat)
    min_lon_lo = floor(min_lon)
    max_lon_hi = ceil(max_lon)

    return min_lat, max_lat, min_lon, max_lon, min_lat_lo, max_lat_hi, min_lon_lo, max_lon_hi

def get_master_date(ctx):
    master_date = ctx.get('master_date', "")
    return master_date

def get_bbox(ctx):
    # min_lat, max_lat, min_lon, max_lon = ctx['region_of_interest']

    if ctx['min_lat'] != "" or ctx['max_lat'] != "" or ctx['min_lon'] != "" or ctx['max_lon'] != "":
        # if any values are present in _context.json we can assume user put them in manually
        bbox_data = get_user_input_bbox(ctx)
    else:
        # if user did not define ANY lat lons
        bbox_data = get_minimum_bounding_rectangle()

    return bbox_data

def download_dem():
    dem_cmd = [
        "{}/applications/dem.py".format(ISCE_HOME), "-a",
        "stitch", "-b", "{} {} {} {}".format(MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI),
        "-r", "-s", "1", "-f", "-c", "|", "tee", "dem.txt"
        #"-n", dem_user, "-w", dem_pass,"-u", dem_url
    ]
    run_cmd(dem_cmd)
    
def get_config_file(run_file, kward):
    config_file = None
    with open(run_file, 'r') as fp:
        line = fp.readline()
        while line:
            if kward in line:
                config_file = line.split(' ')[-1]
                break
            line = fp.readline()
    return config_file.strip()

def get_config_val(config_file, p_name):
    val = None
    with open(config_file, 'r') as fp:
        line = fp.readline()
        while line:
            if p_name in line:
                val = line.split(':')[-1].strip()
                break
            line = fp.readline()
    return val

def get_date_from_str(test_str):
    import re
    temp = re.findall(r'\d+', test_str) 
    res = list(map(int, temp))
    return str(res[0])

def stackSlcDn_run2_5():
    run1File = "./run_files/run_01_unpack_topo_reference"
    run2File = "./run_files/run_02_unpack_secondary_slc"
    runFile = "./run_files/run_02.5_slc_noise_calibration"

    run_cmd("rm -rf {}".format(runFile).split(' '))
    run_cmd("touch {}".format(runFile).split(' '))
    reference_dir = os.path.join(wd, "reference")
    secondary_dir = os.path.join(wd, "secondary")
    refConfig = get_config_file(run1File, "reference")
    print(refConfig)
    
    ref_zip = get_config_val(refConfig, 'dirname')
    print(ref_zip)
    ref_swath = get_config_val(refConfig, 'swaths')
    print(ref_swath)
    
    with open(runFile, 'w') as fw:
        cmd = "/opt/topsstack-hamsar/topsStack/read_calibration_slc.py -zip {} -ext {} -od {} -o -t noise -n '{}'\n".format(ref_zip, bbox, reference_dir, ref_swath)
        fw.write(cmd)
        with open(run2File, 'r') as fp:
            line = fp.readline()
            cmds = []
            while line:
                if 'secondary' in line:
                    secConfig = line.split('-c')[-1].strip()
                    sec_date = get_date_from_str(os.path.basename(secConfig))
                    print("{} : {}".format(sec_date, os.path.basename(secConfig)))
                    sec_zip = get_config_val(secConfig, 'dirname')
                    sec_swath = get_config_val(secConfig, 'swaths')
                    odir=os.path.join(secondary_dir, sec_date )
                    print("{} : {} :{}".format(sec_zip, sec_swath, odir ))
                    cmd = "/opt/topsstack-hamsar/topsStack/read_calibration_slc.py -zip {} -ext {} -od {} -o -t noise -n '{}'\n".format(sec_zip, bbox, odir, sec_swath)
                    fw.write(cmd)
            
                line = fp.readline()
                                                                       

    
import os
import sys
import shutil
import json
import re

from datetime import datetime
from osgeo import ogr, osr

def get_union_polygon_from_bbox(env):
    coords = [
        [ env[3], env[0] ],
        [ env[3], env[1] ],
        [ env[2], env[1] ],
        [ env[2], env[0] ],
    ]
    return {
        "type": "Polygon",
        "coordinates": [ coords ]
    }

# copied from stitch_ifgs.get_union_polygon()
def get_union_polygon(ds_files):
    """
    Get GeoJSON polygon of union of IFGs.
    :param ds_files: list of .dataset.json files, which have the 'location' key
    :return: geojson of merged bbox
    """

    geom_union = None
    for ds_file in ds_files:
        f = open(ds_file)
        ds = json.load(f)
        geom = ogr.CreateGeometryFromJson(json.dumps(ds['location'], indent=2, sort_keys=True))
        if geom_union is None:
            geom_union = geom
        else:
            geom_union = geom_union.Union(geom)
    return json.loads(geom_union.ExportToJson()), geom_union.GetEnvelope()


def get_dataset_met_json_files(cxt):
    """
    returns 2 lists: file paths for dataset.json files and met.json files
    :param cxt: json from _context.json
    :return: list[str], list[str]
    """
    pwd = os.getcwd()
    localize_urls = cxt['localize_slcs']

    met_files, ds_files = [], []
    for localize_url in localize_urls:
        local_path = localize_url['local_path']
        slc_id = local_path.split('/')[0]
        slc_path = os.path.join(pwd, slc_id, slc_id)

        ds_files.append(slc_path + '.dataset.json')
        met_files.append(slc_path + '.met.json')
    return ds_files, met_files


def get_scenes(cxt):
    """
    gets all SLC scenes for the stack
    :param cxt: contents for _context.json
    :return: list of scenes
    """
    localize_urls = cxt['localize_slcs']
    all_scenes = set()
    for localize_url in localize_urls:
        local_path = localize_url['local_path']
        slc_id = local_path.split('/')[0]
        all_scenes.add(slc_id)
    return sorted(list(all_scenes))


def get_min_max_timestamps(scenes_ls):
    """
    returns the min timestamp and max timestamp of the stack
    :param scenes_ls: list[str] all slc scenes in stack
    :return: (str, str) 2 timestamp strings, ex. 20190518T161611
    """
    timestamps = set()

    regex_pattern = r'(\d{8}T\d{6}).(\d{8}T\d{6})'
    for scene in scenes_ls:
        matches = re.search(regex_pattern, scene)
        if not matches:
            raise Exception("regex %s was unable to match with SLC id %s" % (regex_pattern, scene))

        slc_timestamps = (matches.group(1), matches.group(2))
        timestamps = timestamps.union(slc_timestamps)

    min_timestamp = min(timestamps)
    max_timestamp = max(timestamps)
    return min_timestamp.replace('T', ''), max_timestamp.replace('T', '')

def get_min_max_times(scenes_ls):
    """
    returns the min timestamp and max timestamp of the stack
    :param scenes_ls: list[str] all slc scenes in stack
    :return: (str, str) 2 timestamp strings, ex. 20190518T161611
    """
    timestamps = set()

    regex_pattern = r'(\d{8}T\d{6}).(\d{8}T\d{6})'
    for scene in scenes_ls:
        matches = re.search(regex_pattern, scene)
        if not matches:
            raise Exception("regex %s was unable to match with SLC id %s" % (regex_pattern, scene))

        slc_timestamps = (matches.group(1), matches.group(2))
        timestamps = timestamps.union(slc_timestamps)

    min_timestamp = min(timestamps)
    max_timestamp = max(timestamps)
    return min_timestamp, max_timestamp


def create_list_from_keys_json_file(json_files, *args):
    """
    gets all key values in each .json file and returns a sorted array of values
    :param json_files: list[str]
    :return: list[]
    """
    values = set()
    for json_file in json_files:
        if os.path.isfile(json_file): 
            f = open(json_file)
            data = json.load(f)
            for arg in args:
                value = data[arg]
                values.add(value)
    return sorted(list(values))


def camelcase_to_underscore(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()


def get_key_and_convert_to_underscore(json_file_paths, key):
    """
    read through all the json files in file paths, get the first occurrence of key and convert it to underscore
    :param json_file_paths: list[str]
    :param key: str
    :return: key and value
    """
    for json_file in json_file_paths:
        if os.path.isfile(json_file): 
            f = open(json_file)
            data = json.load(f)
            if key in data.keys():
                underscore_key = camelcase_to_underscore(key)
                return underscore_key, data[key]
    return None, None


def generate_dataset_json_data(ctx, version):
    """
    :param cxt: _context.json file
    :param dataset_json_files: list[str] all file paths of SLC's .dataset.json files
    :param version: str: version, ex. v1.0
    :return: dict
    """
    dataset_json_data = dict()
    dataset_json_data['version'] = version

    
    try:      
        dataset_json_data['starttime'], dataset_json_data['endtime']  = get_min_max_times(ctx["localize_slcs"])      
    except Exception as err:
        print(str(err))
    try:
        dataset_json_data['location'] = get_union_polygon_from_bbox(bbox.split(' '))
    except Exception as err:
        print(str(err))
    return dataset_json_data


def generate_met_json_data(ctx, bbox, version):
    """
    :param cxt: _context.json file
    :param met_json_file_paths: list[str] all file paths of SLC's .met.json files
    :param dataset_json_files: list[str] all file paths of SLC's .dataset.json files
    :param version: str: version, ex. v1.0
    :return: dict
    """
    met_json_data = {
        'processing_start': PROCESSING_START,
        'processing_stop': datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
        'version': version
    }

    # generating bbox
    geojson = get_union_polygon_from_bbox(bbox)
    coordinates = geojson['coordinates'][0]
    for coordinate in coordinates:
        coordinate[0], coordinate[1] = coordinate[1], coordinate[0]
    
    met_json_data['bbox'] = bbox
    
    # list of SLC scenes
   
    met_json_data['scenes'] = ctx["localize_slcs"]
    met_json_data['scene_count'] = len(ctx["localize_slcs"])

    # getting timestamps

    met_json_data['sensing_start'], met_json_data['sensing_stop']  = get_min_max_times(ctx["localize_slcs"])

    # additional information
    met_json_data['dataset_type'] = 'stack'

    return met_json_data


def read_context():
    with open('_context.json', 'r') as f:
        cxt = json.load(f)
        return cxt


def create_dataset(bbox):
    VERSION = 'v1.0'
    DATASET_NAMING_TEMPLATE = 'coregistered_slcs-{min_timestamp}-{max_timestamp}'
    PWD = os.getcwd()

    # creating list of all SLC .dataset.json and .met.json files
    ctx = read_context()
    #dataset_json_files, met_json_files = get_dataset_met_json_files(context_json)

    # getting list of SLC scenes and extracting min max timestamp
    slc_scenes = ctx['localize_slcs']
    min_timestamp, max_timestamp = get_min_max_timestamps(slc_scenes)

    # creatin dataset directory
    dataset_name = DATASET_NAMING_TEMPLATE.format(min_timestamp=min_timestamp, max_timestamp=max_timestamp)
    if not os.path.exists(dataset_name):
        os.mkdir(dataset_name)

    # move merged/ master/ slaves/ directory to dataset directory
    move_directories = ['merged', 'reference', 'secondarys']
    for directory in move_directories:
        shutil.move(directory, dataset_name)

    # move _stdout.txt log file to dataset
    shutil.copyfile('_stdout.txt', os.path.join(dataset_name, '_stdout.txt'))

    # generate .dataset.json data
    dataset_json_data = {}
    try:
        dataset_json_data = generate_dataset_json_data(ctx, VERSION)
    except Exception as err:
        print(str(err))
    dataset_json_data['label'] = dataset_name
    print(json.dumps(dataset_json_data, indent=2))

    # generate .met.json data
    met_json_data = generate_met_json_data(ctx, bbox, VERSION)
    print(json.dumps(met_json_data, indent=2))

    # writing .dataset.json to file
    dataset_json_filename = os.path.join(PWD, dataset_name, dataset_name + '.dataset.json')
    with open(dataset_json_filename, 'w') as f:
        json.dump(dataset_json_data, f, indent=2)

    # writing .met.json to file
    met_json_filename = os.path.join(PWD, dataset_name, dataset_name + '.met.json')
    with open(met_json_filename, 'w') as f:
        json.dump(met_json_data, f, indent=2)
        
        
        
def download_slcs(path):
    
    for slc in ctx["localize_slcs"]:
        download_slc(slc, path)       

In [53]:
import xml.etree.ElementTree as ET
import argparse
from argparse import RawTextHelpFormatter
import zipfile
import fnmatch
import re
import os
import sys        
import isce
import iscesys
import isceobj
import isceobj.Sensor.TOPS as TOPS
import isceobj.Sensor.TOPS.BurstSLC as BurstSLC
from isceobj.Image import createImage
import glob
from osgeo import gdal
import numpy as np
from scipy.interpolate import LinearNDInterpolator as interpnd
from iscesys.Parsers import XmlParser



def locateCaliFile(slc,type,polid='vv'):
    swathid = 's1?-iw%d'%(slc.swathNumber)
#    polid = slc.polarization  #default is already set to 'vv'
    print('Using data polarization ', polid)
    match = None
    for dirname in slc.safe:
        match = None
        
        if dirname.endswith('.zip'):
            zf = zipfile.ZipFile(dirname, 'r')
            if type == 'radio':
                pattern = os.path.join('*SAFE','annotation','calibration','calibration-') + swathid + '-slc-' + polid + '*.xml'
                match = fnmatch.filter(zf.namelist(), pattern)
                if (len(match) == 0):
                    raise Exception('No radiometric calibration file found in zip file: {0}'.format(dirname))
                slc.radioCali.append('/vsizip/'+os.path.join(dirname, match[0]) )
                print('Found radiometric calibration files: ', slc.radioCali)
            elif type == 'noise':
                pattern = os.path.join('*SAFE','annotation','calibration','noise-') + swathid + '-slc-' + polid + '*.xml'
                zf = zipfile.ZipFile(dirname, 'r')
                match = fnmatch.filter(zf.namelist(), pattern)
                if (len(match) == 0):
                    raise Exception('No noise calibration file found in zip file: {0}'.format(dirname))
                slc.noiseCali.append('/vsizip/'+os.path.join(dirname, match[0]) )
                print('Found noise calibration files: ', slc.noiseCali)
            zf.close()
        
        else:
            if type == 'radio':
                pattern = os.path.join('annotation','calibration','calibration-') + swathid + '-slc-' + polid + '*.xml'
                match = glob.glob( os.path.join(dirname, pattern))
                if (len(match) == 0):
                    raise Exception('No radiometric calibration file found in {0}'.format(dirname))
                slc.radioCali.append(match[0])
                print('Found radiometric calibration files: ', slc.radioCali)
            elif type == 'noise':
                pattern = os.path.join('annotation','calibration','noise-') + swathid + '-slc-' + polid + '*.xml'
                match = glob.glob( os.path.join(dirname, pattern))
                if (len(match) == 0):
                    raise Exception('No noise calibration file found in {0}'.format(dirname))
                slc.noiseCali.append(match[0])
                print('Found noise calibration files: ', slc.noiseCali)

def sort_caliFiles(slc):
    if len(slc._tiffSrc)>0:
        radioCaliList = []
        noiseCaliList = []
        s1_pat = re.compile(".*(S1[AB]_.+?_\d{4}\d{2}\d{2}T.*).zip.*")
        for swath in range(len(slc._tiffSrc)):
            match = s1_pat.search(slc._tiffSrc[swath])
            if match:
                id_prefix=match.groups(1)[0]
            else:
                raise Exception('Unable to extract date in multi-slice scene: %s' %  slc._tiffSrc[swath])
            for slc_ind in range(len(slc.radioCali)):
                if id_prefix in slc.radioCali[slc_ind]:
                    radioCaliList.append(slc.radioCali[slc_ind])
       	        if id_prefix in slc.noiseCali[slc_ind]:
                    noiseCaliList.append(slc.noiseCali[slc_ind])
        slc.radioCali=radioCaliList
        slc.noiseCali=noiseCaliList        
       
    
def write2flt( data, lat, lon, outName, nanvalue=-9999.90039062 ):
    width = data.shape[1]
    length = data.shape[0]
    data[ (data == nanvalue) ] = 0
    outm = np.matrix(data,np.float32)
    gpmFile = open(outName, "wb")
    outm.tofile(gpmFile)
    gpmFile.close();

def write2xml( data, xFirst, yFirst, dx, dy, outName, projection='lat/lon'):
    length  = data.shape[0] 
    width   = data.shape[1] 
    xStep   = dx 
    yStep   = dy 
    proj    = projection
    xmlName = outName + '.xml'
    xmldict  = {'METADATA_LOCATION':xmlName,
                'data_type':'Float',
                'image_type':'BIL',
                'Coordinate1':{'size':width,'startingValue':xFirst,'delta':xStep},
                'Coordinate2':{'size':length,'startingValue':yFirst,'delta':yStep},
                'FILE_NAME':outName,
                'number_bands':1    }

    demImage = createImage()
    demImage.init(xmldict)
    demImage.renderHdr()

def readXMLDict(FileName):
    xml = XmlParser
    xml = XmlParser.XmlParser()
    tmpdict  = xml.parse(FileName)[0]
    xmldict = dict((k.replace(" ", "").lower(), value) for (k, value) in tmpdict.items())
    if xmldict == "":
        print("No dictionay loaded from "+FileName+". \n");
    else:
        return(xmldict);    

    
'''
To run radiometric and thermal noise calibration on a zip file, using vh-pol data, within a given extent:

read_calibration_slc.py -zip S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.zip -od 20150315 -ext 34.6 34.65 -79.08 -78.97 -o -p vh -t noise

To run radiometric calibration only using the topsApp.xml and scene xml file (using default vv-pol):

read_calibration_slc.py -i topsApp.py -is s1_20150315.xml -o -t radio 

To output the radiometric calibration file only

read_calibration_slc.py -zip S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.zip -od 20150315 -oc

To output the radiometric and thermal noise calibration file (radio + noise) within a given extent

read_calibration_slc.py -zip S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.zip -od 20150315 -o -ext 34.6 34.65 -79.08 -78.97 -t noise

'''
#read_calibration_slc.py( -zip {} -ext {} -od {} -o -t noise -n '{}'\n".format(ref_zip, bbox, reference_dir, ref_swath)
def read_calibration_slc(fin, ext, odir, swath_num, type="noise", output=True, pol="vv", ck=False, oc=False): 

    ext = [np.float(ext[x]) for x in range(len(ext))]    
    if type not in ('radio','noise'):
        error('ERROR: type (-t) needs to be either radio or noise')
    try:
        swathList = swath_num.split()
    except:
        swathList = [1, 2, 3]

    for swath in swathList:
        slc = TOPS.createSentinel1()
        slc.polarization = pol
        slc.safe = fin
        slc.swathNumber = int(swath)
        #slc.product = TOPS.createTOPSSwathSLCProduct()
        slc.regionOfInterest = ext
        slc.product.bursts = iscesys.Component.createTraitSeq('bursts')
        slc.output = os.path.join(odir,'IW{0}'.format(swath))
        try:
            slc.parse()
        except Exception as err:
            print('Could not extract swath {0} from {1}'.format(swath, slc.safe))
            print('Generated: ', err)
            continue
        #slc.parse()
        if len(slc.product.bursts) == 0:
            continue
        slc.radioCali = []
        slc.noiseCali = []
        locateCaliFile(slc,'radio',pol) 
        locateCaliFile(slc,'noise',pol) 
        sort_caliFiles(slc)
        if slc._numSlices == 1:
            slc.radioCali = (slc.product.numberOfBursts) * [slc.radioCali[0]]
            slc.noiseCali = (slc.product.numberOfBursts) * [slc.noiseCali[0]]
    
        ################# Output calibrated SLC filesi (-o option)  ######################
        if output:  # generate raw and calibrated slc

            width  = slc._burstWidth
            length = slc._burstLength
  
            ####Check if aux file corrections are needed
            useAuxCorrections = False
            if ('002.36' in slc.product.processingSoftwareVersion) and (slc.auxFile is not None):
                useAuxCorrections = True
  
            ###If not specified, for single slice, use width and length from first burst
            if width is None:
                width = slc.product.bursts[0].numberOfSamples
  
            if length is None:
                length = slc.product.bursts[0].numberOfLines

            if os.path.isdir(slc.output):
                print('Output directory exists. Overwriting ...')
            else:
                print('Creating directory {0} '.format(slc.output))
                os.makedirs(slc.output)

            prevTiff  = None
            prevRadio = None
            prevNoise = None
            for index, burst in enumerate(slc.product.bursts):

                print('############################################')
                print('#  burst {} of IW{} '.format(index+1,swath))
                print('############################################')
                ####tiff for single slice
                if (len(slc._tiffSrc) == 0) and (len(slc.tiff)==1):
                    tiffToRead  = slc.tiff[0]
                    radioToRead = slc.radioCali[0]
                    noiseToRead = slc.noiseCali[0] 
                else: ##tiffSrc for multi slice
                    tiffToRead  = slc._tiffSrc[index]
                    radioToRead = slc.radioCali[index]
                    noiseToRead = slc.noiseCali[index]
                ###To minimize reads and speed up 
                if tiffToRead != prevTiff:
                    src=None
                    band=None
                    src = gdal.Open(tiffToRead, gdal.GA_ReadOnly)
                    fullWidth = src.RasterXSize
                    fullLength = src.RasterYSize
                    band = src.GetRasterBand(1)
                    prevTiff  = tiffToRead
                    xlist  = list(range(fullWidth))
                    ylist  = list(range(fullLength))
                    fullX, fullY = np.meshgrid(xlist, ylist)

                #### Thermal Noise Calibration (Optional)
                if ( noiseToRead != prevNoise ) and ( type == 'noise' ):
                    if noiseToRead.startswith('/vsizip'): # read from zip file
                       parts = noiseToRead.split(os.path.sep)
                       if parts[2] == '':
                           parts[2] = os.path.sep
                       zipname = os.path.join(*(parts[2:-4]))
                       fname = os.path.join(*(parts[-4:]))
                       if not os.path.isfile(zipname):
                           print('File ',zipname,' does not exist.')
                           sys.exit(1)
                       zf = zipfile.ZipFile(zipname, 'r')
                       xmlstr = zf.read(fname)
                    else: # Read out calibration from xml file
                        fid = open(noiseToRead)
                        xmlstr = fid.read()
                    ### Extract the calibration LUT
                    root   = ET.fromstring(xmlstr)
                    try:
                        caliRoot = root.find('noiseVectorList')
                        nVectors  = int(caliRoot.items()[0][1])
                    except:
                        caliRoot = root.find('noiseRangeVectorList')
                        nVectors  = int(caliRoot.items()[0][1])
                    xxn = [];
                    yyn = [];
                    zzn = [];
                    for child in caliRoot.getchildren():
                        line  = int(child.find('line').text)
                        pixel = list(map(int, child.find('pixel').text.split()))
                        nPixel = int(child.find('pixel').items()[0][1])
                        try:
                            noiseLut = list(map(float, child.find('noiseLut').text.split()))
                        except:
                            noiseLut = list(map(float, child.find('noiseRangeLut').text.split()))
                        xxn = xxn + pixel
                        yyn = yyn + [line]*nPixel
                        zzn = zzn + noiseLut
                    if ck:
                        if not any(zzn):  # if all-zeros in zzn
                            zf.close()
                            for izip in range(len(fin)):
                                print("All-zero noise LUT in: {}".format(fin[izip]))
                                if(os.path.islink(fin[izip])):
                                    print("{} is a symlink, doing unlink fo SLC with bad LUT".format(fin[izip]))
                                    os.unlink(fin[izip])
                                else:
                                    print('{} is not a symlink. Not removing file. Please run this again with symlinked files'.format(fin[izip]))
                        else:
                            print('LUT is okay for :', fin, '. Doing nothing.')
                            sys.exit(1)
                    else:    
                        npt = len(zzn)
                        coordn = np.hstack((np.array(xxn).reshape(npt,1),np.array(yyn).reshape(npt,1)))
                        noise  = np.array(zzn).reshape(npt,1)
                        print('Start 2D interpolation on noiseLut. This may take a while....')
                        interpfn2 = interpnd(coordn,noise)
                        noiseIntrp = interpfn2(fullX, fullY)
                        prevNoise = noiseToRead
                        if oc:
                            ocfile = os.path.join(odir,'IW{0}'.format(swath),'noise-iw'+str(swath)+'.cal')
                            write2flt( noiseIntrp, ylist, xlist, ocfile )
                            write2xml( noiseIntrp, ylist[0], xlist[0], 1, 1, ocfile ) 
                            print('Writing noise calibration file to ', ocfile)

                #### Radio Calibration
                if radioToRead != prevRadio:
                    ### Locate radiometric calibration xml file
                    if radioToRead.startswith('/vsizip'): # read from zip file
                        parts = radioToRead.split(os.path.sep)
                        if parts[2] == '':
                            parts[2] = os.path.sep
                        zipname = os.path.join(*(parts[2:-4]))
                        fname = os.path.join(*(parts[-4:]))
                        if not os.path.isfile(zipname):
                            print('File ',zipname,' does not exist.')
                            sys.exit(1)
                        zf = zipfile.ZipFile(zipname, 'r')
                        xmlstr = zf.read(fname)
                    else: # Read out calibration from xml file
                        fid = open(radioToRead)
                        xmlstr = fid.read()
                    ### Extract the calibration LUT
                    root   = ET.fromstring(xmlstr)
                    caliRoot = root.find('calibrationVectorList')
                    nVectors  = int(caliRoot.items()[0][1])
                    xx = [];
                    yy = [];
                    zz = [];
                    for child in caliRoot.getchildren():
                        line  = int(child.find('line').text)
                        pixel = list(map(int, child.find('pixel').text.split()))
                        nPixel = int(child.find('pixel').items()[0][1])
                        sigmaNought = list(map(float, child.find('sigmaNought').text.split()))
                        xx = xx + pixel
                        yy = yy + [line]*nPixel
                        zz = zz + sigmaNought
                    if ck:
                        if not any(zz):  # if all-zeros in zz
                            zf.close()
                            for izip in range(len(fin)):
                                os.unlink(fin[izip]) 
                            print('All-zero radiometric calibration LUT. Remove the symbolic link file ',fin )
                    else: 
                        npt = len(zz)
                        coord = np.hstack((np.array(xx).reshape(npt,1),np.array(yy).reshape(npt,1)))
                        sigma  = np.array(zz).reshape(npt,1)
                        print('Start 2D interpolation on sigmaNought. This may take a while....')
                        interpfn1  = interpnd(coord,sigma)
                        sigmaIntrp = interpfn1(fullX, fullY)
                        prevRadio  = radioToRead
                        if oc:   # read in calibration file
                            ocfile = os.path.join(odir,'IW{0}'.format(swath),'radio-iw'+str(swath)+'.cal')
                            write2flt( sigmaIntrp, ylist, xlist, ocfile )
                            write2xml( sigmaIntrp, ylist[0], xlist[0], 1, 1, ocfile ) 
                            print('Writing radiometric calibration file to ', ocfile)

                if ck:
                    sys.exit(0)

                #outfile1 = os.path.join(slc.output, 'burst_%02d_bk'%(index+1) + '.slc')
                outfile2 = os.path.join(slc.output, 'burst_%02d'%(index+1) + '.slc')
 
                ####Use burstnumber to look into tiff file
                ####burstNumber still refers to original burst in slice
                lineOffset = (burst.burstNumber-1) * burst.numberOfLines
                data = band.ReadAsArray(0, lineOffset, burst.numberOfSamples, burst.numberOfLines)
                datacrop = data[burst.firstValidLine:burst.lastValidLine, burst.firstValidSample:burst.lastValidSample]
 
                ######Saving the radar cross-section
                #if not os.path.isfile(outfile1):
                #    fid = open(outfile1, 'wb')
                #    outdata  = np.zeros((length,width), dtype=np.complex64)  #output radar cross-section
                #    sigmaraw = np.square(abs(datacrop))
                #    outdata[burst.firstValidLine:burst.lastValidLine, burst.firstValidSample:burst.lastValidSample] = sigmaraw
                #    outdata.tofile(fid)
                #    fid.close()
                #    ####Render ISCE XML
                #    slcImage = isceobj.createSlcImage()
                #    slcImage.setByteOrder('l')
                #    slcImage.setFilename(outfile1)
                #    slcImage.setAccessMode('read')
                #    slcImage.setWidth(width)
                #    slcImage.setLength(length)
                #    slcImage.setXmin(0)
                #    slcImage.setXmax(width)
                #    slcImage.renderHdr()
                #    burst.image = slcImage
                #else:
                #    print('File {0} already exist. Skip file output.'.format(outfile1))
    
                fid = open(outfile2, 'wb')
                outdata = np.zeros((length,width), dtype=np.complex64)  # convert to radar cross-section domain
                calicrop = np.squeeze(sigmaIntrp[burst.firstValidLine:burst.lastValidLine, burst.firstValidSample:burst.lastValidSample])
                sigmacali = np.square(abs(datacrop.real + 1j*datacrop.imag)/calicrop)
                if type == 'radio':
                    recomb = np.sqrt(sigmacali)*np.exp(1j*np.angle(datacrop))
                elif type == 'noise':  #This means radio + noise
                    noiseCrop = np.squeeze(noiseIntrp[burst.firstValidLine:burst.lastValidLine, burst.firstValidSample:burst.lastValidSample])
                    noiseCorr = noiseCrop/np.square(calicrop)
                    [ind0,ind1]   = np.where( abs(datacrop) == 0 )
                    sigmadn       = sigmacali - noiseCorr
                    sigmadn[ind0,ind1] = 0
                    [ind2,ind3]   = np.where( sigmadn < 0 )
                    sigmadn[ind2,ind3] = 1e-9  #instead of clipping at 0, assign a minimal value
                    recomb    = np.sqrt(abs(sigmadn))*np.exp(1j*np.angle(datacrop))
                outdata[burst.firstValidLine:burst.lastValidLine, burst.firstValidSample:burst.lastValidSample] = recomb
                #Skip the correction for the Elevation Antenna Pattern for now 
                outdata.tofile(fid)
                fid.close()
                ####Render ISCE XML
                slcImage = isceobj.createSlcImage()
                slcImage.setByteOrder('l')
                slcImage.setFilename(outfile2)
                slcImage.setAccessMode('read')
                slcImage.setWidth(width)
                slcImage.setLength(length)
                slcImage.setXmin(0)
                slcImage.setXmax(width)
                slcImage.renderHdr()
            src=None
            band=None

def run_stackSlcDn_run2_5():
    run1File = "./run_files/run_01_unpack_topo_reference"
    run2File = "./run_files/run_02_unpack_secondary_slc"



    reference_dir = os.path.join(wd, "reference")
    secondary_dir = os.path.join(wd, "secondary")
    refConfig = get_config_file(run1File, "reference")
    print(refConfig)
    
    ref_zip = get_config_val(refConfig, 'dirname')
    print(ref_zip)
    ref_swath = get_config_val(refConfig, 'swaths')
    print(ref_swath)
    

    try:
        #read_calibration_slc(fin, ext, odir, swath_num, type="noise", output=True, pol="vv", ck=False, oc=False): 
        read_calibration_slc(ref_zip, bbox, reference_dir, ref_swath)
        with open(run2File, 'r') as fp:
            line = fp.readline()
            while line:
                if 'secondary' in line:
                    secConfig = line.split('-c')[-1].strip()
                    sec_date = get_date_from_str(os.path.basename(secConfig))
                    print("{} : {}".format(sec_date, os.path.basename(secConfig)))
                    sec_zip = get_config_val(secConfig, 'dirname')
                    sec_swath = get_config_val(secConfig, 'swaths')
                    odir=os.path.join(secondary_dir, sec_date )
                    print("{} : {} :{}".format(sec_zip, sec_swath, odir ))
                    read_calibration_slc(sec_zip, bbox, odir, sec_swath)
            
                line = fp.readline()
    except Exception as e:
        print(str(e))

Using default ISCE Path: /opt/isce2/isce


In [13]:
MINLAT, MAXLAT, MINLON, MAXLON, MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI =get_bbox(ctx)
print("{} {} {} {} {} {} {} {}".format(MINLAT, MAXLAT, MINLON, MAXLON, MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI))

bbox = "{} {} {} {}".format(MINLAT, MAXLAT, MINLON, MAXLON)
print(bbox)

34.6002832 34.6502392 -79.0801608 -78.9705888 34 35 -80 -78
34.6002832 34.6502392 -79.0801608 -78.9705888


**Download Sentinel-1 data SLC**

The SLCs identified in the parameters cell are downloaded. Note that these files are significant in size (around 5GB each) which will collectively require a significant amount of time to download (expect around 2 minutes per SLC).

In [18]:
path = SLC_PATH
download_slcs(path)

***Download Orbit Files based on SLC***

The orbit files identified based on the set of SLCs downloaded in the previous step. They are placed in the orbits subdirectory.

In [35]:
get_orbit_files()


slc_start_dt_str: 2015-03-15T23:13:19
slc_end_dt_str: 2015-03-15T23:13:49
checking 2015-03-15
slc_start_dt_str: 2015-08-18T23:13:26
slc_end_dt_str: 2015-08-18T23:13:56
checking 2015-08-18
slc_start_dt_str: 2015-08-30T23:13:32
slc_end_dt_str: 2015-08-30T23:14:02
checking 2015-08-30
slc_start_dt_str: 2016-04-14T23:13:21
slc_end_dt_str: 2016-04-14T23:13:50
checking 2016-04-14
slc_start_dt_str: 2016-04-14T23:13:48
slc_end_dt_str: 2016-04-14T23:14:16


**Download DEM data and generate DEM file**
Similarly to the orbit files, the DEM information to retain is computed based on the bounding box.


In [36]:
download_dem()

['/opt/isce2/isce/applications/dem.py', '-a', 'stitch', '-b', '34 35 -80 -78', '-r', '-s', '1', '-f', '-c', '|', 'tee', 'dem.txt']
Calling : /opt/isce2/isce/applications/dem.py -a stitch -b 34 35 -80 -78 -r -s 1 -f -c | tee dem.txt
b'% Total    % Received % Xferd  Average Speed   Time    Time     Time  Current'
b'Dload  Upload   Total   Spent    Left  Speed'
b'0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100   267  100   267    0     0   1420      0 --:--:-- --:--:-- --:--:--  1420'
b'1 7534k    1 81920    0     0    99k      0  0:01:15 --:--:--  0:01:15   99k\r100 7534k  100 7534k    0     0  5290k      0  0:00:01  0:00:01 --:--:-- 11.7M'
b'% Total    % Received % Xferd  Average Speed   Time    Time     Time  Current'
b'Dload  Upload   Total   Spent    Left  Speed'
b'0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100   267  100   267    0     0   2153      0 --:--:-- --:--:-- --:--:--  2153'
b'0     0    0     0    0     

**Fix paths in dem file**

This step invokes the isce2 application fixImageXML.py in order to fill in the complete path to the files referenced in the dem file.


In [37]:

import os
cwd = os.getcwd()
print("cwd : {}".format(cwd))
if os.path.exists("dem.txt"):
    cmd = ["awk", "'/wgs84/ {print $NF;exit}'", "dem.txt"]
    WGS84 = run_cmd_output(cmd).decode("utf-8").strip()
    wgs84_file = os.path.join(cwd, WGS84)
    print("WGS84 : a{}b".format(wgs84_file))
    if os.path.exists(wgs84_file):
        print("Found wgs84 file: {}".format(wgs84_file))
        fix_cmd = ["{}/applications/fixImageXml.py".format(ISCE_HOME), "--full", "-i", "{}".format(wgs84_file) ]
        run_cmd(fix_cmd) 
    else:
        print("NO WGS84 FILE FOUND : {}".format(wgs84_file))

cwd : /home/jovyan/topsstack-hamsar
Calling: awk '/wgs84/ {print $NF;exit}' dem.txt
WGS84 : a/home/jovyan/topsstack-hamsar/demLat_N34_N35_Lon_W080_W078.dem.wgs84b
Found wgs84 file: /home/jovyan/topsstack-hamsar/demLat_N34_N35_Lon_W080_W078.dem.wgs84
['/opt/isce2/isce/applications/fixImageXml.py', '--full', '-i', '/home/jovyan/topsstack-hamsar/demLat_N34_N35_Lon_W080_W078.dem.wgs84']
Calling : /opt/isce2/isce/applications/fixImageXml.py --full -i /home/jovyan/topsstack-hamsar/demLat_N34_N35_Lon_W080_W078.dem.wgs84
b'Using default ISCE Path: /opt/isce2/isce'
b'Writing geotrans to VRT for /home/jovyan/topsstack-hamsar/demLat_N34_N35_Lon_W080_W078.dem.wgs84'


#### AUX_CAL file download ####

The following calibration auxliary (AUX_CAL) file is used for **antenna pattern correction** to compensate the range phase offset of SAFE products with **IPF verison 002.36** (mainly for images acquired before March 2015). If all your SAFE products are from another IPF version, then no AUX files are needed. Check [ESA document](https://earth.esa.int/documents/247904/1653440/Sentinel-1-IPF_EAP_Phase_correction) for details. 

Run the command below to download the AUX_CAL file once and store it somewhere (_i.e._ ~/aux/aux_cal) so that you can use it all the time, for `stackSentinel.py -a` or `auxiliary data directory` in `topsApp.py`.

```
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
```

In [ ]:
%%bash
mkdir -p ./AuxDir
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ --directory ./AuxDir
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ


The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

`stackSentinel.py` generates all configuration and run files required to be executed on a stack of Sentinel-1 TOPS data. When stackSentinel.py is executed for a given workflow (-W option) a **configs** and **run_files** folder is generated. No processing is performed at this stage. Within the run_files folder different run\_#\_description files are contained which are to be executed as shell scripts in the run number order. Each of these run scripts call specific configure files contained in the “configs” folder which call ISCE in a modular fashion. The configure and run files will change depending on the selected workflow. To make run_# files executable, change the file permission accordingly (e.g., `chmod +x run_01_unpack_slc`).

```bash
stackSentinel.py -H     #To see workflow examples,
stackSentinel.py -h     #To get an overview of all the configurable parameters
```

Required parameters of stackSentinel.py include:

```cfg
-s SLC_DIRNAME          #A folder with downloaded Sentinel-1 SLC’s. 
-o ORBIT_DIRNAME        #A folder containing the Sentinel-1 orbits. Missing orbit files will be downloaded automatically
-a AUX_DIRNAME          #A folder containing the Sentinel-1 Auxiliary files
-d DEM_FILENAME         #A DEM (Digital Elevation Model) referenced to wgs84
```

In the following, different workflow examples are provided. Note that stackSentinel.py only generates the run and configure files. To perform the actual processing, the user will need to execute each run file in their numbered order.

In all workflows, coregistration (-C option) can be done using only geometry (set option = geometry) or with geometry plus refined azimuth offsets through NESD (set option = NESD) approach, the latter being the default. For the NESD coregistrstion the user can control the ESD coherence threshold (-e option) and the number of overlap interferograms (-O) to be used in NESD estimation.


In [38]:
%%bash
rm -rf /home/jovyan/topsstack-hamsar/run_files 

In [39]:
master_date=get_master_date(ctx)
print("master_date : {}".format(master_date))

if master_date:
    print("MASTER_DATE exists:".format(master_date) )   
    cmd = [
        "/opt/isce2/src/isce2/contrib/stack/topsStack/stackSentinel.py",  "-s", "zip/", "-d", "{}".format(wgs84_file), "-a", "AuxDir/", "-m", "{}".format(master_date), "-o", "orbits", 
        "-b", "\"{} {} {} {}\"".format(MINLAT, MAXLAT, MINLON, MAXLON), 
        "-W", "slc", "-C", "geometry"
    ]
else:
    print("MASTER_DATE DOES NOT EXIST")          
    cmd = [
        "/opt/isce2/src/isce2/contrib/stack/topsStack/stackSentinel.py",  "-s", "zip/", "-d", "{}".format(wgs84_file), "-a", "AuxDir/", "-o", "orbits", 
        "-b", "\"{} {} {} {}\"".format(MINLAT, MAXLAT, MINLON, MAXLON), 
        "-W", "slc", "-C", "geometry"
    ]
run_cmd(cmd)

master_date : 
MASTER_DATE DOES NOT EXIST
['/opt/isce2/src/isce2/contrib/stack/topsStack/stackSentinel.py', '-s', 'zip/', '-d', '/home/jovyan/topsstack-hamsar/demLat_N34_N35_Lon_W080_W078.dem.wgs84', '-a', 'AuxDir/', '-o', 'orbits', '-b', '"34.6002832 34.6502392 -79.0801608 -78.9705888"', '-W', 'slc', '-C', 'geometry']
Calling : /opt/isce2/src/isce2/contrib/stack/topsStack/stackSentinel.py -s zip/ -d /home/jovyan/topsstack-hamsar/demLat_N34_N35_Lon_W080_W078.dem.wgs84 -a AuxDir/ -o orbits -b "34.6002832 34.6502392 -79.0801608 -78.9705888" -W slc -C geometry
b'Using default ISCE Path: /opt/isce2/isce'
b'Number of SAFE files found: 5'
b'self.start_date_time 2016-04-14 23:13:21'
b'orbit_start_date_time : 2015-03-15 23:00:44'
b'self.stop_date_time : 2016-04-14 23:13:50'
b'orbit_stop_date_time : 2015-03-17 00:58:44'
b'did not match'
b'self.start_date_time 2016-04-14 23:13:21'
b'orbit_start_date_time : 2016-04-13 23:00:43'
b'self.stop_date_time : 2016-04-14 23:13:50'
b'orbit_stop_date_time :

**stackSlcDn_run2_5**

An intermediate run file must be manually added between run files 2 and 3 in order to perform radiometric and thermal noise cancellation. This step generates this run file, executing against each SLC file

In [40]:
%%bash
cmd="/opt/topsstack-hamsar/topsStack/stackSlcDn_run2.5.sh $MINLAT $MAXLAT $MINLON $MAXLON"
echo "Making read_calibration_slc.py runfile: $cmd"
eval $cmd

Making read_calibration_slc.py runfile: /opt/topsstack-hamsar/topsStack/stackSlcDn_run2.5.sh    
msterConfig :  /home/jovyan/topsstack-hamsar/configs/config_reference
fzip : /home/jovyan/topsstack-hamsar/zip/S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.zip
writing  ./run_files/run_02.5_slc_noise_calibration


**run_01_unpack_slc_topo_reference:**
Unpacks the reference SLC files using ISCE readers, also unpackaging the antenna elevation pattern correction file if necessary. This run file also produces the reference geometry files that are consumed downstream.

Includes commands to unpack Sentinel-1 TOPS SLCs using ISCE readers. For older SLCs which need antenna elevation pattern correction, the file is extracted and written to disk. For newer version of SLCs which don’t need the elevation antenna pattern correction, only a gdal virtual “vrt” file (and isce xml file) is generated. The “.vrt” file points to the Sentinel SLC file and reads them whenever required during the processing. If a user wants to write the “.vrt” SLC file to disk, it can be done easily using gdal_translate (e.g. gdal_translate –of ENVI File.vrt File.slc). 
The “run_01_unpack_slc_topo_reference” also includes a command that refers to the config file of the stack reference, which includes configuration for running topo for the stack reference. Note that in the pair-wise processing strategy one should run topo (mapping from range-Doppler to geo coordinate) for all pairs. However, with stackSentinel, topo needs to be run only one time for the reference in the stack. 


In [41]:
%%bash
echo "## STEP 1 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh
conda activate base


start=`date +%s`
echo "sh run_files/run_01_unpack_topo_reference"
sh run_files/run_01_unpack_topo_reference
end=`date +%s`
runtime1=$((end-start))
echo "STEP 1 RUN TIME : $runtime1"

## STEP 1 ##
sh run_files/run_01_unpack_topo_reference
Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/topsstack-hamsar/zip/S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.zip', '--swaths', '1 2 3', '--orbitdir', '/home/jovyan/topsstack-hamsar/orbits', '--outdir', '/home/jovyan/topsstack-hamsar/reference', '--auxdir', '/home/jovyan/topsstack-hamsar/AuxDir', '--bbox', '34.6002832 34.6502392 -79.0801608 -78.9705888', '--pol', 'vv']
Input XML files:  ['/vsizip//home/jovyan/topsstack-hamsar/zip/S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.zip/S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.SAFE/annotation/s1a-iw1-slc-vv-20150315t231319-20150315t231347-005049-006569-004.xml']
Input TIFF files:  ['/vsizip//home/jovyan/topsstack-hamsar/zip/S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664.zip/S1A_IW_SLC__1SDV_20150

/opt/isce2/src/isce2/contrib/stack/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':


**run_02_unpack_secondary_slc:**

Unpack Secondary SLCs
In a manner similar to the SLCs in Run File 01 above, this run file unpacks the secondary SLCs from each of the input SLC zip files.

In [42]:
%%bash
echo "## STEP 2 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh

start=`date +%s`
Num=`cat run_files/run_02_unpack_secondary_slc | wc | awk '{print $1}'`
echo $Num
echo "cat run_files/run_02_unpack_secondary_slc | parallel -j2 --eta --load 50%"
cat run_files/run_02_unpack_secondary_slc | parallel -j2 --eta --load 50%
end=`date +%s`

runtime2=$((end-start))
echo STEP 2 RUN TIME : $runtime2

## STEP 2 ##
3
cat run_files/run_02_unpack_secondary_slc | parallel -j2 --eta --load 50%
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/topsstack-hamsar/zip/S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_93D5.zip', '--swaths', '1 2 3', '--orbitdir', '/home/jovyan/topsstack-hamsar/orbits', '--outdir', '/home/jovyan/topsstack-hamsar/secondarys/20150818', '--auxdir', '/home/jovyan/topsstack-hamsar/AuxDir', '--bbox', '34.6002832 34.6502392 -79.0801608 -78.9705888', '--pol', 'vv']
Input XML files:  ['/vsizip//home/jovyan/topsstack-hamsar/zip/S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_93D5.zip/S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_93D5.SAFE/annotation/s1a-iw1-slc-vv-20150818t231326-20150818t231354-007324-00a0e0-004.xml']
Input TIFF files:  ['/vsizip//home/jovyan/topsstack-hamsar/zip/S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_9

sh: /dev/tty: No such device or address

Computers / CPU cores / Max jobs to run
1:local / 8 / 2

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  

#### run_02.5_slc_noise_calibration ###
This run file, which was manually introduced in step 19, runs radiometric and thermal noise calibration using vh-pol data against the SLCs.

In [44]:
%%bash
echo "## STEP 2.5 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:~/topsstack-hamsar/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:~/topsstack-hamsar/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh
conda activate base

start=`date +%s`
echo "cat run_files/run_02.5_slc_noise_calibration | parallel -j2 --eta --load 50%"
cat run_files/run_02.5_slc_noise_calibration | parallel -j2 --eta --load 50%
end=`date +%s`

runtime2x5=$((end-start))
echo STEP 2.5 RUN TIME : $runtime2x5

## STEP 2.5 ##
cat run_files/run_02.5_slc_noise_calibration | parallel -j2 --eta --load 50%
STEP 2.5 RUN TIME : 1


sh: /dev/tty: No such device or address

Computers / CPU cores / Max jobs to run
1:local / 8 / 2

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 4 AVG: 0.00s  local:2/0/100%/0.0s usage: read_calibration_slc.py [-h] [-i FXML] [-is FXMLSCENE]
                               [-zip FIN [FIN ...]] [-od ODIR] [-o]
                               [-ext EXT [EXT ...]] [-n SWATH_NUM] [-p POL]
                               [-t TYPE] [-oc] [-ck]
read_calibration_slc.py: error: argument -ext/--crop_extent: expected at least one argument
ETA: 0s Left: 3 AVG: 0.00s  local:1/1/100%/0.0s usage: read_calibration_slc.py [-h] [-i FXML] [-is FXMLSCENE]
                               [-zip FIN [FIN ...]] [-od ODIR] [-o]
                               [-ext EXT [EXT ...]] [-n SWATH_NUM] [-p POL]
                               [-t TYPE] [-oc] [-ck]
read_calibration_slc.py: error: argument -ext/--crop_extent: expected at least one argument
ETA: 0s Left: 2 AVG: 0

**run_03_average_baseline:**

Computes average baseline for the stack. These baselines are not used for processing anywhere. They are only an approximation and can be used for plotting purposes.

In [45]:
%%bash
echo "## STEP 3 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh
conda activate base

start=`date +%s`
echo "cat run_files/run_03_average_baseline | parallel -j2 --eta --load 50%"
cat run_files/run_03_average_baseline | parallel -j2 --eta --load 50%
end=`date +%s`
runtime3=$((end-start))
echo STEP 3 RUN TIME : $runtime3

## STEP 3 ##
cat run_files/run_03_average_baseline | parallel -j2 --eta --load 50%
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/home/jovyan/topsstack-hamsar/reference/', '--secondary', '/home/jovyan/topsstack-hamsar/secondarys/20150818', '--baseline_file', '/home/jovyan/topsstack-hamsar/baselines/20150315_20150818/20150315_20150818.txt']
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
Bprep:  [-45.32065630019591, -45.280996955720966]
Bpar:  [32.46790969040835, 32.59920593051198]
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
Bprep:  [-42.65769351249399, -42.6074168582937]
Bpar:  [35.88432444382608, 36.012829251875694]
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Runni

sh: /dev/tty: No such device or address

Computers / CPU cores / Max jobs to run
1:local / 8 / 2
sh: /dev/tty: No such device or address

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s /opt/isce2/src/isce2/contrib/stack/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 0s Left: 2 AVG: 0.00s  local:1/1/100%/2.0s /opt/isce2/src/isce2/contrib/stack/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 0s Left: 1 AVG: 0.00s  local:1/2/100%/1.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 1 AVG: 0.50s  local:1/2/100%/1.5s /opt/isce2/src/isce2/contrib/stack/topsStack/SentinelWrapper.py:113: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if iString is '':
ETA: 0s Left: 0 AVG: 0.33s  

**STEP 4 : run_04_fullBurst_geo2rdr:**

Full Burst geo2rdr
This run file estimates geometrical offsets between secondary burst overlaps and the stack reference burst overlaps. The secondaries are then resampled to the stack reference burst overlaps.


In [46]:
%%bash
echo "## STEP 4 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh
conda activate base

start=`date +%s`

echo "cat run_files/run_04_fullBurst_geo2rdr  | parallel -j2 --eta --load 50%"
cat run_files/run_04_fullBurst_geo2rdr  | parallel -j2 --eta --load 50%
end=`date +%s`
runtime4=$((end-start))
echo STEP 4 RUN TIME : $runtime4

## STEP 4 ##
cat run_files/run_04_fullBurst_geo2rdr  | parallel -j2 --eta --load 50%
API open (WR): /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818/IW1/range_01.off
API open (WR): /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818/IW1/azimuth_01.off
GDAL open (R): /home/jovyan/topsstack-hamsar/geom_reference/IW1/hgt_01.rdr.vrt
GDAL open (R): /home/jovyan/topsstack-hamsar/geom_reference/IW1/lat_01.rdr.vrt
GDAL open (R): /home/jovyan/topsstack-hamsar/geom_reference/IW1/lon_01.rdr.vrt
API close:  /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818/IW1/range_01.off
API open (R): /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818/IW1/range_01.off
API close:  /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818/IW1/range_01.off
API close:  /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818/IW1/azimuth_01.off
API open (R): /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818/IW1/azimuth_01.off
API close:  /home/jovyan/topsstack-hamsar/coreg_secondarys/20150818

sh: /dev/tty: No such device or address

Computers / CPU cores / Max jobs to run
1:local / 8 / 2

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  

**STEP 5 : run_05_fullBurst_resample:**

Using orbit and DEM data, this run file computes geometrical offsets among all secondary SLCs and the stack reference. These offsets, with the misregistration time series are used for precise coregistration of each burst SLC.

In [47]:
%%bash
echo "## STEP 5 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh
conda activate base

start=`date +%s`
echo "cat run_files/run_05_fullBurst_resample  | parallel -j2 --eta --load 50%"
cat run_files/run_05_fullBurst_resample  | parallel -j2 --eta --load 50%
end=`date +%s`
runtime5=$((end-start))
echo STEP 5 RUN TIME : $runtime5


## STEP 5 ##
cat run_files/run_05_fullBurst_resample  | parallel -j2 --eta --load 50%
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/home/jovyan/topsstack-hamsar/secondarys/20150818', '--reference', '/home/jovyan/topsstack-hamsar/reference', '--coregdir', '/home/jovyan/topsstack-hamsar/coreg_secondarys/20150818', '--azimuth_misreg', '/home/jovyan/topsstack-hamsar/misreg/azimuth/dates/20150818.txt', '--range_misreg', '/home/jovyan/topsstack-hamsar/misreg/range/dates/20150818.txt']
Estimated burst offset:  0
Shifts:  {0: 0, 1: -1}
Chi squared: 0.000026
Misfit radians - Max: 8.527249701728579e-05 , Min : -0.0001572892342665 
True
2021-03-09 22:42:47,747 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_LINES has been set to the default value 1494 which is the number of lines in the slc image.
2021-03-09 22:42:47,748 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_WIDTH has been set to the defa

sh: /dev/tty: No such device or address

Computers / CPU cores / Max jobs to run
1:local / 8 / 2

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 3 AVG: 0.00s  

**STEP 6 : run_06_timeseries_misreg:**

This run file extracts the valid region between burst SLCs at the overlap area of the bursts. This region changes slightly for different acquisitions and must be retained for merging the bursts to eliminate invalid data.


In [48]:
%%bash
echo "## STEP 6 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh
conda activate base

start=`date +%s`
echo "sh run_files/run_06_extract_stack_valid_region"
sh run_files/run_06_extract_stack_valid_region
end=`date +%s`
runtime6=$((end-start))
echo STEP 6 RUN TIME : $runtime6

## STEP 6 ##
sh run_files/run_06_extract_stack_valid_region
creating  /home/jovyan/topsstack-hamsar/stack
checking the number of bursts in coreg_secondarys against the one in reference
all secondary images have the same number of bursts as the reference
******************
swath:  1
/home/jovyan/topsstack-hamsar/coreg_secondarys/20150818
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
/home/jovyan/topsstack-hamsar/coreg_secondarys/20150830
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
/home/jovyan/topsstack-hamsar/coreg_secondarys/20160414
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
writing  /home/jovyan/topsstack-hamsar/stack/IW1.xml
******************
swath:  2
/home/jovyan/topsstack-hamsar/coreg_secondarys/20150818
minSecondary,maxSecondary 1 2
minReference,maxReference 1 2
minBurst, maxBurst:  1 2
/home/jovyan/topsstack-hamsar/coreg_secondarys/20150830
minSecondary,ma

**run_07_geo2rdr_resample: Merge Reference Secondary SLCs**

This run file merges all bursts for the reference and coregistered SLCs. Geometry files are also merged.
Using orbit and DEM, geometrical offsets among all secondary SLCs and the stack reference is computed. The geometrical offsets, together with the misregistration time-series (from previous step) are used for precise coregistration of each burst SLC. 

In [49]:
%%bash
echo "## STEP 7 ##"
source /opt/isce2/isce_env.sh
export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}
source /opt/conda/etc/profile.d/conda.sh
conda activate base

start=`date +%s`

FILE=run_files/run_07_merge
if test -f "$FILE"; then
    echo "cat run_files/run_07_merge  | parallel -j2 --eta --load 50%"
    cat run_files/run_07_merge | parallel -j2 --eta --load 50%
else
    echo "cat run_files/run_07_merge_reference_secondary_slc  | parallel -j2 --eta --load 50%"
    cat run_files/run_07_merge_reference_secondary_slc | parallel -j2 --eta --load 50%
fi

end=`date +%s`
runtime7=$((end-start))
echo STEP 7 RUN TIME : $runtime7

## STEP 7 ##
cat run_files/run_07_merge_reference_secondary_slc  | parallel -j2 --eta --load 50%
Input file size is 45670, 3724
0...10...20...30...40...50...60...70...80...90...100 - done.
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/topsstack-hamsar/stack', '--inp_reference', '/home/jovyan/topsstack-hamsar/reference', '--dirname', '/home/jovyan/topsstack-hamsar/geom_reference', '--name_pattern', 'lat*rdr', '--outfile', '/home/jovyan/topsstack-hamsar/merged/geom_reference/lat.rdr', '--method', 'top', '--multilook', '--range_looks', '9', '--azimuth_looks', '3', '--multilook_tool', 'gdal', '--no_data_value', '0']
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 2
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 2
writing merged file to disk ...
True
multi looking using gdal ...
/home/jovyan/topsstack-hamsar/merged/geom_ref

sh: /dev/tty: No such device or address

Computers / CPU cores / Max jobs to run
1:local / 8 / 2

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AVG: 0.00s  local:2/0/100%/0.0s sh: /dev/tty: No such device or address
ETA: 0s Left: 10 AV

**Create Dataset**
The final step ‘packages’ up the results into a single sub-directory named as coregistered_slcs-<start_date>-<end_date> and generates json files describing the dataset (e.g. the polygon  coordinates of the bounding box) and associated metadata (e.g. included SLCs).

In [50]:
create_dataset(bbox)


{
  "version": "v1.0",
  "starttime": "20150315T231319",
  "endtime": "20160414T231416",
  "location": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          "-78.9705888",
          "34.6002832"
        ],
        [
          "-78.9705888",
          "34.6502392"
        ],
        [
          "-79.0801608",
          "34.6502392"
        ],
        [
          "-79.0801608",
          "34.6002832"
        ]
      ]
    ]
  },
  "label": "coregistered_slcs-20150315231319-20160414231416"
}
{
  "processing_start": "2021-03-09T20:27:46",
  "processing_stop": "2021-03-09T22:48:33",
  "version": "v1.0",
  "bbox": "34.6002832 34.6502392 -79.0801608 -78.9705888",
  "scenes": [
    "S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664",
    "S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_93D5",
    "S1A_IW_SLC__1SDV_20150830T231332_20150830T231402_007499_00A5A9_02B3",
    "S1A_IW_SLC__1SDV_20160414T231321_20160414T231350_010824_01030B_F02B",
 

In [51]:
run_stop_time = get_current_time()
print("PGE run stop time : {}".format(run_stop_time))
total_run_time = run_stop_time - run_start_time
print("Total Run Time = {}".format(total_run_time))

PGE run stop time : 2021-03-09 22:48:33.821992
Total Run Time = 2:21:33.076985
